In [5]:
import os
from modules.merge_dataset import merge_dataset
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner
from modules import pipeline, components

In [6]:
merge_dataset("data/movie.csv", "data/rating.csv")

'Merge dataset success'

In [7]:
PIPELINE_NAME = "movie-recommender-pipeline"

# pipeline inputs
DATA_ROOT = "data/merge"
TRANSFORM_MODULE_FILE = "modules/transform.py"
TUNER_MODULE_FILE = "modules/tuner.py"
TRAINER_MODULE_FILE = "modules/trainer.py"

# pipeline outputs
OUTPUT_BASE = "outputs"

serving_model_dir = os.path.join(OUTPUT_BASE, "serving_model")
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")

In [8]:
components_args = {
    "data_dir": DATA_ROOT,
    "transform_module": TRANSFORM_MODULE_FILE,
    "tuner_module": TUNER_MODULE_FILE,
    "trainer_module": TRAINER_MODULE_FILE,
    "train_steps": 1000,
    "eval_steps": 400,
    "serving_model_dir": serving_model_dir,
    "epochs": 5,
}

components = components.init_components(components_args)

pipeline = pipeline.init_pipeline(
    pipeline_root,
    PIPELINE_NAME,
    metadata_path,
    components,
)

BeamDagRunner().run(pipeline)

INFO:tensorflow:Assets written to: outputs\movie-recommender-pipeline\Transform\transform_graph\35\.temp_path\tftransform_tmp\a65e158f57444a279e9f925d6da1fa9f\assets


INFO:tensorflow:Assets written to: outputs\movie-recommender-pipeline\Transform\transform_graph\35\.temp_path\tftransform_tmp\a65e158f57444a279e9f925d6da1fa9f\assets


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.
